# Tag 2. Kapitel 8. Daten-Manipulationen

## Lektion 44.  Guide zur Verwendung von tidyr

Wir kennen jetzt bereits `dplyr` und seine Funktionen. Zusätzlich können wir uns jetzt `tidyr` aneignen. Dies ist ein komplementäres Paket, das uns dabei hilft, ordentliche Datensätze zu erstellen. Was meinen wir überhaupt wenn wir "ordentlicher" Datensatz sagen?

Ordentliche Daten liegen vor, wenn jede Zeile eine eigene Beobachtung darstellt und jede Spalte eine Variable. Durch diese Organisation der Daten beinhaltet jede Zelle genau den Wert für eine Variable für eine Beobachtung. Durch diese Format können wir Daten leichter verstehen, analysieren und visualisieren.

Nach dieser Lektion könnt ihr euch auf folgendes Cheatsheet beziehen: [Data Wrangling](http://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf)

## Installation von tidyr

In [86]:
install.packages('tidyr',repos = 'http://cran.us.r-project.org')
# Evtl. data.table installieren
install.packages("data.table")

Warning message:
"package 'tidyr' is in use and will not be installed"Warning message:
"package 'data.table' is in use and will not be installed"

In [88]:
library(tidyr)
library(data.table)

## Data.frames versus data.tables

Alle data.tables sind auch data.frames. Vereinfacht gesagt sind data.tables data.frames die extra Features bieten. 

data.frame ist in der Installation von R inbegriffen.

data.table ist ein Paket, dass data.frame erweitert. Zwei der wichtigsten Features sind Geschwindigkeit und saubere Syntax.

Jedoch ist die Syntax der data.table etwas unterschiedlich zur Standard R Syntax von data.frame. Diese Unterschiede können für ein untrainiertes Auge allerdings schwer zu erkennen sein. Wenn man mit einem Code-Abschnitt arbeitet, der nicht anzeigt, dass mit data.tables gearbeitet wird, und diesen auf einen Data Frame anwenden möchte, dann kann es zu Errors kommen.

Um einige der Unterschiede festzuhalten:

* viel schnellerer und sehr intuitive **by** Operationen
* Wir werden nicht ausversehen riesige Data Frames ausgeben, die wir mit Strg-C stoppen müssten. data.table verhindert solche Vorfälle
* schnellere und einfacheres Lesen von Datein mit **fread**
* das Paket beinhaltet weitere Funktionen wie %between% oder rbindlist
* und insgesamt einfach etwas schnellere Grundfunktionen

# Nutzung von tidyr

Wir werden einige der nützlichsten Funktionen in tidyr abdecken. Dazu gehören die folgenden:

* gather()
* spread()
* separate()
* unite()

## Beispiel Datensatz

Lass uns jetzt einige Daten erzeugen, die mit tidyr ordentlich gemacht werden müssen

In [89]:
firma <- c(1,1,1,2,2,2,3,3,3)
jr <- c(1998,1999,2000,1998,1999,2000,1998,1999,2000)
q1 <- runif(9, min=0, max=100)
q2 <- runif(9, min=0, max=100)
q3 <- runif(9, min=0, max=100)
q4 <- runif(9, min=0, max=100)

df <- data.frame(firma=firma,jahr=jr,Qtr1 = q1,Qtr2 = q2,Qtr3 = q3,Qtr4 = q4)

In [90]:
df

firma,jahr,Qtr1,Qtr2,Qtr3,Qtr4
1,1998,31.24142,64.35533,15.103400,84.05206
1,1999,17.05111,34.88872,26.342010,45.00045
1,2000,14.54117,14.71333,6.414722,97.07631
2,1998,73.57931,75.81704,92.919413,88.97070
2,1999,42.08913,25.10458,34.529138,15.35714
2,2000,11.43164,50.19196,43.868852,16.83074
3,1998,49.17495,73.39670,44.732164,75.35942
3,1999,96.18519,72.92747,67.636679,55.11637
3,2000,68.16501,43.82732,7.909525,81.83202


# Gather() und Spread()

Manchmal vergleichen Leute diese Operationen mit Pivot-Tabellen in Excel. Schauen wir uns einige Beispiele an:

## gather()

Die `gather()` Funktion fasst mehrere Spalten in Schlüssel-Paar Werte zusammen. Der Data Frame von oben würde als "breit" bezeichnet werden, da jedes Quartal eine eigene Spalte bzw. Variable hat. Um diese Zeitdimension neu zu ordnen, können wir alle Quartale in eine Spalte vereinen. Außerdem können wir dann alle Werte, die mit den einzelnen Quartalen asoziiert sind, in eine zweite Spalte schreiben.

In [91]:
# Unter Verwendung des Pipe Operators
df %>% gather(Quartal,Umsatz,Qtr1:Qtr4) %>% head

firma,jahr,Quartal,Umsatz
1,1998,Qtr1,31.24142
1,1999,Qtr1,17.05111
1,2000,Qtr1,14.54117
2,1998,Qtr1,73.57931
2,1999,Qtr1,42.08913
2,2000,Qtr1,11.43164


In [92]:
# Nur mit der Funktion
head(gather(df,Quartal,Umsatz,Qtr1:Qtr4))

firma,jahr,Quartal,Umsatz
1,1998,Qtr1,31.24142
1,1999,Qtr1,17.05111
1,2000,Qtr1,14.54117
2,1998,Qtr1,73.57931
2,1999,Qtr1,42.08913
2,2000,Qtr1,11.43164


## spread()

Ist das Gegenstück zu `gather()`:

In [93]:
# Step 1.
aktien <- data.frame(
  datum = as.Date('2009-01-01') + 0:9,
  X = rnorm(10, 0, 1),
  Y = rnorm(10, 0, 2),
  Z = rnorm(10, 0, 4)
)
aktien

datum,X,Y,Z
2009-01-01,1.10509159,2.0340756,-5.0715554
2009-01-02,0.05493291,2.3002347,-3.0552499
2009-01-03,-0.53703429,3.9320139,-1.0288409
2009-01-04,-0.81873941,-0.4090300,0.4213357
2009-01-05,1.02677054,-0.2470760,-6.2706810
2009-01-06,-0.52386910,3.1813537,-7.0499903
2009-01-07,0.75928170,0.2255228,-2.0214215
2009-01-08,0.31538345,0.4337674,-6.7384930
2009-01-09,0.35598608,0.2968420,0.5154720
2009-01-10,0.69384972,3.4733101,6.5739083


In [94]:
# Step 2.
# Daten als Key-Value Paare darstellen
aktienm <- aktien %>% gather("aktie", "preis", -datum)
aktienm # 30 x 3 Datensatz

datum,aktie,preis
2009-01-01,X,1.10509159
2009-01-02,X,0.05493291
2009-01-03,X,-0.53703429
2009-01-04,X,-0.81873941
2009-01-05,X,1.02677054
2009-01-06,X,-0.52386910
2009-01-07,X,0.75928170
2009-01-08,X,0.31538345
2009-01-09,X,0.35598608
2009-01-10,X,0.69384972


In [95]:
# Step 3.
# Key-Value Datensätze über Aktie darstellen
aktienm %>% spread("aktie", "preis")

datum,X,Y,Z
2009-01-01,1.10509159,2.0340756,-5.0715554
2009-01-02,0.05493291,2.3002347,-3.0552499
2009-01-03,-0.53703429,3.9320139,-1.0288409
2009-01-04,-0.81873941,-0.4090300,0.4213357
2009-01-05,1.02677054,-0.2470760,-6.2706810
2009-01-06,-0.52386910,3.1813537,-7.0499903
2009-01-07,0.75928170,0.2255228,-2.0214215
2009-01-08,0.31538345,0.4337674,-6.7384930
2009-01-09,0.35598608,0.2968420,0.5154720
2009-01-10,0.69384972,3.4733101,6.5739083


In [96]:
# Step 3.
# Key-Value Datensätze über Datum darstellen
aktienm %>% spread("datum", "preis")

aktie,2009-01-01,2009-01-02,2009-01-03,2009-01-04,2009-01-05,2009-01-06,2009-01-07,2009-01-08,2009-01-09,2009-01-10
X,1.105092,0.05493291,-0.5370343,-0.8187394,1.026771,-0.5238691,0.7592817,0.3153835,0.3559861,0.6938497
Y,2.034076,2.30023473,3.9320139,-0.4090300,-0.247076,3.1813537,0.2255228,0.4337674,0.2968420,3.4733101
Z,-5.071555,-3.05524994,-1.0288409,0.4213357,-6.270681,-7.0499903,-2.0214215,-6.7384930,0.5154720,6.5739083


# Separate und Unite

## separate()

Ausgehend von einer Regular Expression oder einem Vektor von Zeichenpositionen kann `separate()` eine einzelne Spalte in mehrere aufteilen:

In [97]:
v <- c(NA, "a.x", "b.y", "c.z")
df <- data.frame(v)
v
df

[1] NA    "a.x" "b.y" "c.z"

v
NA
a.x
b.y
c.z


In [98]:
df %>% separate(v, c("col1", "col2"))

col1,col2
NA,NA
a,x
b,y
c,z


## unite()

`unite()` ist üblicherweise die Funktion um mehrere Spalten zu einer zusammenzufassen:

In [99]:
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


In [100]:
mtcars %>%
  unite(vs_am, vs, am) %>% head

,mpg,cyl,disp,hp,drat,wt,qsec,vs_am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0_1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0_1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1_1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1_0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0_0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1_0,3,1


In [101]:
# separate() ist das Gegenstück zu unite()
# vergleichen Sie 
mtcars %>%
  unite(vs_am, vs, am) %>%
  separate(vs_am, c("vs", "am")) %>% head

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


In [102]:
# ... mit mtcars
mtcars %>% head

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


Herzlichen Glückwunsch! Sie sind mit Lektion 44. fertig!